#### <mark>**Problem 1**</mark>

MDP Definition
An MDP is defined by the tuple: (S, A, P, R, γ)

* S is State Space (S)
* A is Action Space (A)
* P is a state transition probability matrix 
$$P_{ss'} = \mathbb{P}[S_{t+1} = s' \mid S_t = s, A_t = a]$$
* R is a reward function 
$$R_s = \mathbb{E}[R_{t+1} \mid S_t = s, A_t = a]$$
* γ is a discount factor
$$\gamma \in [0, 1]$$

___

**The State Space (S)**
The state must provide a complete snapshot of the system's physical configuration so the agent can predict the outcome of its actions.
* Joint Positions: The current position of each linkage (angles or displacements).
* Joint Velocities: Necessary to account for momentum and ensure smoothness (penalizing jerk or acceleration).
* End-Effector Position: The (x, y, z) coordinates of the "hand" relative to the target object (position + orientation). End-effector pose can be derived from joint states, but including it improves learning stability.
* Object Status: Position of the object to be picked.
* Gripper Status: Whether the gripper has successfully secured the object (open/closed or force)

Reasoning: 
* Without velocity data, the agent cannot perceive "smoothness" or deceleration, leading to jerky movements or overshooting.
* Explicit goal information allows generalization across different pick-and-place locations.
* Positions + velocities fully describe the arm’s dynamics.

___

**The Action Space (A)**
Because we want fast and smooth movements, actions should be continuous and low-level. We avoid discrete steps (like Up, Down, Right, Left).
* Torque Control (τ): The agent sends a specific amount of electric current/force to each motor at every time step: $$a_t = \tau_t$$
    - Pros:
        1. Maximum control authority
        2. Learns smooth, dynamic motions

    - Cons: Harder to learn, requires more data

* Alternative (Velocity Control): Specifying desired angular velocities, though torque control is the "gold standard" for truly fluid, human-like motion.

    - Pros: 
        1. More stable learning
        2. Still supports smooth movements

Reasoning: 
Controlling torques allows the agent to utilize gravity and inertia, which is essential for high-speed efficiency.

___

**The Reward Function (R)**
The reward function is the most critical part of the design. It must balance competing goals: accuracy, speed, and mechanical preservation. A composite reward function might look like this:
$$R = W_1(R_{\text{task}}) - W_2(R_{\text{energy}}) - W_3(R_{\text{jerk}})$$

Objectives: 
1. Reach the target accurately (accuracy)
2. Do it quickly (Speed)
3. Move smoothly and avoid excessive forces or unsafe behaviour

| Component | Definition | Reasoning |
| :--- | :--- | :--- |
| **Task Success** | Large positive reward for placing the object; negative penalty based on distance to target. | Drives the agent toward the primary goal. |
| **Energy Penalty** | A penalty proportional to the square of the torques applied. | Encourages efficiency and prevents "over-powering" the motors. |
| **Smoothness (Jerk)** | A penalty on the derivative of acceleration (change in torque). | **Crucial for smoothness.** It penalizes sudden, vibrating movements that cause wear and tear. |
| **Time Penalty** | A small negative reward for every step taken. | Incentivizes the "fast" requirement. |
___

**The Transition Dynamics (P) and Environment**
* Transition Function: $$\mathbb{P}(s_{t+1} \mid s_t, a_t)$$
* Time Step (∆t): Must be very small (e.g., 1ms to 10ms). High-speed control requires a high control frequency to react to physical oscillations.
* Physics Engine: The environment would likely be a simulator like MuJoCo or Isaac Gym, which models gravity, friction, and Coriolis forces before deploying to a physical arm.

___

**Episode Termination**
An episode ends when one of the following items is true:
* Object successfully placed in the target place
* Or the maximum time steps are exceeded 
* Or safety constraint violated (collision, joint limit)

___

**Discount factor γ**
* Typically $$\gamma \in [0.95, 0.99]$$
* Encourages planning smooth trajectories rather than greedy moves
* High discount factor helps capture delayed rewards from smooth control

___

#### <mark>**Problem 2**</mark>

**Setup and Assumptions**
To perform Value Iteration, we use the Bellman Optimality Equation.The update rule for a state $s$ at iteration $k+1$ is:$$V_{k+1}(s) = \max_{a} \left( R(s, a) + \gamma \sum_{s'} P(s' | s, a) V_k(s') \right)$$

* Rewards(R): The problem states $R(s)$ is constant for all actions, so $R(s, a) = R(s)$.
    - $R(s1) = 5$
    - $R(s2) = 10$
    - $R(s3) = 1$
    - $R(s4) = 2$

* Transitions: Deterministic. If an action moves into a wall, $s' = s$.
* Discount Factor ($\gamma$): A discount factor is required for the values to converge in a continuing task. Since one was not specified in the prompt, I will assume a standard value of $\gamma = 0.9$.
* Initialization: We initialize the value of all states to 0.$$V_0(s) = 0 \text{ for all } s$$
* Note on Initial Policy: Value Iteration searches for the optimal policy directly by updating values. It does not require an initial policy (unlike Policy Iteration). Therefore, the "Initial Policy $\pi(up|s)=1$" is ignored for the initialization step.

____

**Iteration 1**
In the first iteration, we update the values using $V_0$ (which is all zeros).
1. Equation ApplicationSince $V_0(s') = 0$ for all states, the term $\gamma V_0(s')$ becomes 0. The equation simplifies to just the immediate reward:$$V_1(s) = \max_{a} [R(s) + 0] = R(s)$$
2. Value Function UpdatesState s1: $V_1(s1) = 5$State s2: $V_1(s2) = 10$State s3: $V_1(s3) = 1$State s4: $V_1(s4) = 2$
3. Updated Value Function ($V_1$)

| State | V1​(s) | 
| :--- | :----: | 
| s1    | 5.0   |
| s2    | 10.0  |
| s3    | 1.0   |
| s4    | 2.0   |

___

**Iteration 2**
Now we calculate $V_2$ using the values from $V_1$.

Formula: $V_{2}(s) = R(s) + 0.9 \times \max_{a} V_1(s')$.

Value Function Updates (Step-by-Step)

State s1 (R=5):
* Up: Hits wall $\to$ stays in s1 ($V_1=5$). Value: $5 + 0.9(5) = 9.5$
* Down: Moves to s3 ($V_1=1$). Value: $5 + 0.9(1) = 5.9$
* Left: Hits wall $\to$ stays in s1 ($V_1=5$). Value: $5 + 0.9(5) = 9.5$
* Right: Moves to s2 ($V_1=10$). Value: $5 + 0.9(10) = 14.0$
* Max: The best action is Right.
* Update: $$V_2(s1) = 14.0$$

State s2 (R=10):
* Up: Hits wall $\to$ stays in s2 ($V_1=10$). Value: $10 + 0.9(10) = 19.0$
* Down: Moves to s4 ($V_1=2$). Value: $10 + 0.9(2) = 11.8$
* Left: Moves to s1 ($V_1=5$). Value: $10 + 0.9(5) = 14.5$
* Right: Hits wall $\to$ stays in s2 ($V_1=10$). Value: $10 + 0.9(10) = 19.0$
* Max: The best actions are Up or Right.
* Update: $$V_2(s2) = 19.0$$

State s3 (R=1):
* Up: Moves to s1 ($V_1=5$). Value: $1 + 0.9(5) = 5.5$
* Down: Hits wall $\to$ stays in s3 ($V_1=1$). Value: $1 + 0.9(1) = 1.9$
* Left: Hits wall $\to$ stays in s3 ($V_1=1$). Value: $1 + 0.9(1) = 1.9$
* Right: Moves to s4 ($V_1=2$). Value: $1 + 0.9(2) = 2.8$
* Max: The best action is Up.
* Update: $$V_2(s3) = 5.5$$

State s4 (R=2):
* Up: Moves to s2 ($V_1=10$). Value: $2 + 0.9(10) = 11.0$
* Down: Hits wall $\to$ stays in s4 ($V_1=2$). Value: $2 + 0.9(2) = 3.8$
* Left: Moves to s3 ($V_1=1$). Value: $2 + 0.9(1) = 2.9$
* Right: Hits wall $\to$ stays in s4 ($V_1=2$). Value: $2 + 0.9(2) = 3.8$
* Max: The best action is Up.
* Update: $$V_2(s4) = 11.0$$

The Value Function ($V$) after Second Iteration

| State | $V_2(s)$ | Optimal Action (Greedy Policy) |
| :--- | :--- | :--- |
| **s1** | 14.0 | Right |
| **s2** | 19.0 | Up / Right |
| **s3** | 5.5 | Up |
| **s4** | 11.0 | Up |

#### <mark>**Problem 3**</mark>

In [ ]:
import numpy as np
from gridworld_mdp import GridWorldMDP
from value_iteration import standard_value_iteration, inplace_value_iteration

def print_results(mdp, V, policy, title):
    print(f"\n--- {title} ---")
    print("Optimal Value Function (V*):")
    print(np.round(V, 2))
    print("\nOptimal Policy (π*):")
    for r in range(mdp.rows):
        row_str = " | ".join([f"{str(policy[r, c]):^3}" for c in range(mdp.cols)])
        print(f"| {row_str} |")
    print("-" * 30)

def main():
    mdp = GridWorldMDP(gamma=0.9, theta=1e-4)

    # --- Task 1: Standard VI ---
    print("Running Standard Value Iteration...")
    V_std, iters_std, time_std = standard_value_iteration(mdp)
    pi_std = mdp.get_optimal_policy(V_std)
    print_results(mdp, V_std, pi_std, "Standard Value Iteration")

    # --- Task 2: In-Place VI ---
    print("\nRunning In-Place Value Iteration...")
    V_in, iters_in, time_in = inplace_value_iteration(mdp)
    pi_in = mdp.get_optimal_policy(V_in)
    print_results(mdp, V_in, pi_in, "In-Place Value Iteration")

    # --- Comparison ---
    print("\n=== Performance Comparison ===")
    print(f"{'Method':<25} | {'Iterations':<10} | {'Time (s)':<10}")
    print("-" * 50)
    print(f"{'Standard VI':<25} | {iters_std:<10} | {time_std:.6f}")
    print(f"{'In-Place VI':<25} | {iters_in:<10} | {time_in:.6f}")

if __name__ == "__main__":
    main()

Running Standard Value Iteration...

--- Standard Value Iteration ---
Optimal Value Function (V*):
[[-0.43  0.63  1.81  3.12  4.58]
 [ 0.63  1.81  3.12  4.58  6.2 ]
 [ 1.81  3.12  4.58  6.2   8.  ]
 [ 3.12  4.58  6.2   8.   10.  ]
 [ 4.58  6.2   8.   10.    0.  ]]

Optimal Policy (π*):
|  →  |  →  |  →  |  ↓  |  ↓  |
|  →  |  →  |  →  |  →  |  ↓  |
|  →  |  ↓  |  →  |  →  |  ↓  |
|  →  |  →  |  →  |  →  |  ↓  |
|  →  |  →  |  →  |  →  |  G  |
------------------------------

Running In-Place Value Iteration...

--- In-Place Value Iteration ---
Optimal Value Function (V*):
[[-0.43  0.63  1.81  3.12  4.58]
 [ 0.63  1.81  3.12  4.58  6.2 ]
 [ 1.81  3.12  4.58  6.2   8.  ]
 [ 3.12  4.58  6.2   8.   10.  ]
 [ 4.58  6.2   8.   10.    0.  ]]

Optimal Policy (π*):
|  →  |  →  |  →  |  ↓  |  ↓  |
|  →  |  →  |  →  |  →  |  ↓  |
|  →  |  ↓  |  →  |  →  |  ↓  |
|  →  |  →  |  →  |  →  |  ↓  |
|  →  |  →  |  →  |  →  |  G  |
------------------------------

=== Performance Comparison ===
Method     